# 1.Import packages

In [13]:
# Importing all required packages at the start of the notebook
import IPython

from qiime2 import Visualization

import qiime2 as q2
import pandas as pd
import matplotlib.pyplot as plt
import os

%matplotlib inline

# 2.Import the data

In [14]:
# Location of the projects data
!mkdir -p "Project_data"
data_dir = "Project_data/Import_and_Denoising"

In [15]:
%%bash -s $data_dir
mkdir -p "$1"

wget -nc --progress=dot:giga -P "$1" https://polybox.ethz.ch/index.php/s/uV06vmm96ZzB5eM/download/fungut_forward_reads.qza

chmod -R +rxw "$1"

File ‘Project_data/Import_and_Denoising/fungut_forward_reads.qza’ already there; not retrieving.



# 3.Feature table construction

In [4]:
# Visual summary of the data
! qiime demux summarize \
    --i-data $data_dir/fungut_forward_reads.qza \
    --o-visualization $data_dir/fungut_forward_reads_demux_seqs.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: Project_data/Import_and_Denoizing/fungut_forward_reads_demux_seqs.qzv


In [11]:
Visualization.load(f"{data_dir}/fungut_forward_reads_demux_seqs.qzv")

<visualization: Visualization uuid: b5307e0f-14b1-47f6-b251-c3ef573d4d2e>

The mean quality score is quite high along the nucleotides positions (quality score = 38 at the last position). However, at an early position, the lower whisker drops drastically, suggesting heterogeneity in quality among the different reads.
Because of this, and because ITS have a variable size, we decided to first filter based on PHRED quality score, and then to not use a length truncation.

# 4. Quality filtering

In [6]:
! qiime quality-filter q-score \
    --i-demux $data_dir/fungut_forward_reads.qza \
    --p-min-quality 30 \
    --o-filtered-sequences $data_dir/fungut_forward_reads_quality_filtered.qza \
    --o-filter-stats $data_dir/quality_filtering_stats.qza 

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved SampleData[SequencesWithQuality] to: Project_data/Import_and_Denoizing/fungut_forward_reads_quality_filtered.qza
Saved QualityFilterStats to: Project_data/Import_and_Denoizing/quality_filtering_stats.qza


In [7]:
! qiime metadata tabulate \
    --m-input-file $data_dir/quality_filtering_stats.qza \
    --o-visualization $data_dir/quality_filtering_stats.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: Project_data/Import_and_Denoizing/quality_filtering_stats.qzv


In [5]:
Visualization.load(f"{data_dir}/quality_filtering_stats.qzv")

<visualization: Visualization uuid: d6d7f5e7-a0d2-4bc4-8dc3-2d0c864d1d9b>

In [6]:
! qiime demux summarize \
    --i-data $data_dir/fungut_forward_reads_quality_filtered.qza \
    --o-visualization $data_dir/fungut_forward_reads_filtered_demux_seqs.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: Project_data/Import_and_Denoising/fungut_forward_reads_filtered_demux_seqs.qzv


In [7]:
Visualization.load(f"{data_dir}/fungut_forward_reads_filtered_demux_seqs.qzv")

<visualization: Visualization uuid: b9e1c482-e5ec-4045-a3b6-1db3df616929>

# 5. Denoising

In [9]:
! qiime dada2 denoise-single \
    --i-demultiplexed-seqs $data_dir/fungut_forward_reads_quality_filtered.qza \
    --p-trunc-len 0 \
    --p-n-threads 3 \
    --o-table $data_dir/dada2_table.qza \
    --o-representative-sequences $data_dir/dada2_rep_set.qza \
    --o-denoising-stats $data_dir/dada2_stats.qza

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved FeatureTable[Frequency] to: Project_data/Import_and_Denoizing/dada2_table.qza
Saved FeatureData[Sequence] to: Project_data/Import_and_Denoizing/dada2_rep_set.qza
Saved SampleData[DADA2Stats] to: Project_data/Import_and_Denoizing/dada2_stats.qza


In [10]:
! qiime metadata tabulate \
  --m-input-file $data_dir/dada2_stats.qza \
  --o-visualization $data_dir/dada2_stats.qzv

! qiime feature-table tabulate-seqs \
  --i-data $data_dir/dada2_rep_set.qza \
  --o-visualization $data_dir/dada2_rep_set.qzv

! qiime feature-table summarize \
  --i-table $data_dir/dada2_table.qza \
  --m-sample-metadata-file Project_data/Metadata/updated_fungut_metadata.tsv \
  --o-visualization $data_dir/dada2_table.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: Project_data/Import_and_Denoizing/dada2_stats.qzv
/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: Project_data/Import_and_Denoizing/dada2_rep_set.qzv
/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources 

In [8]:
Visualization.load(f"{data_dir}/dada2_stats.qzv")

<visualization: Visualization uuid: 380bb22d-74ec-4475-879c-a59fe7c7b395>

In [9]:
Visualization.load(f"{data_dir}/dada2_rep_set.qzv")

<visualization: Visualization uuid: a7047582-4652-438b-9c20-f88c01f0c920>

In [10]:
Visualization.load(f"{data_dir}/dada2_table.qzv")

<visualization: Visualization uuid: 0ce75ba2-867d-4fcd-8375-55d87ab11c5c>